In [63]:
from collections import defaultdict
import math
import numpy as np
import pandas as pd
from typing import List

#@title Debugging
# See https://zohaib.me/debugging-in-google-collab-notebook/ for tips,
# as well as docs for pdb and ipdb.
DEBUG = False #@param {type:"boolean"}
GDRIVE_BASE = "/content/drive" #@param
DATAFRAME_PATH = "/MyDrive/amazon_rainforest_files/monthly_large.csv" #@param
RASTER_BASE = "/MyDrive/amazon_rainforest_files/amazon_rasters/" #@param
MODEL_SAVE_LOCATION = "/MyDrive/amazon_rainforest_files/dnn_model.h5" #@param

def get_dataframe_path_from_params() -> str:
  root = GDRIVE_BASE if GDRIVE_BASE else ""
  return f"{root}{DATAFRAME_PATH}"

def get_model_save_location() -> str:
  root = GDRIVE_BASE if GDRIVE_BASE else ""
  return f"{root}{MODEL_SAVE_LOCATION}"

def get_raster_path_from_params(filename) -> str:
  root = GDRIVE_BASE if GDRIVE_BASE else ""
  return f"{root}{RASTER_BASE}{filename}"

In [ ]:
# Access data stored on Google Drive
if GDRIVE_BASE:
    from google.colab import drive
    drive.mount(GDRIVE_BASE)

if DEBUG:
    %pip install -Uqq ipdb
    import ipdb
    %pdb on

In [148]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from matplotlib import pyplot as plt

def kl_divergence_loss(real, predicted):
    real_value = real[0]
    real_variance = real[1] + 0.00000001
    predicted_value = predicted[0]
    predicted_variance = predicted[1] + 0.00000001

    kl_loss = -0.5 + tf.math.log(predicted_variance/real_variance) + \
     (tf.square(real_variance) + tf.square(real_value - predicted_value))/ \
     2*tf.square(predicted_variance)
    return tf.math.reduce_mean(kl_loss)

def train_nn(
        X: pd.DataFrame,
        Y: pd.DataFrame,
        hidden_layers: List[int],
        epochs: int,
        batch_size: int):
    # Layers share between mean and variance regressors.
    shared_layers = []
    for num_nodes in hidden_layers:
        shared_layers.append(layers.Dense(
            num_nodes,
            activation='sigmoid'))

    #Initialize input layers and connect them to shared layers.
    num_inputs = X.shape[1]
    inputs = keras.Input(shape=(num_inputs,))
    x = inputs
    for shared_layer in shared_layers:
        x = shared_layer(x)

    # Output is variance and mean, and connect to shared nodes.
    mean_output_layer = layers.Dense(1, activation='linear', name='mean_output')
    mean_output_node = mean_output_layer(x)
    variance_output_layer = layers.Dense(1, activation='sigmoid', name='variance_output')
    variance_output_node = variance_output_layer(x)
    outputs = [mean_output_node, variance_output_node]

    model = keras.Model(inputs=inputs, outputs=outputs)

    optimizer = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss={'mean_output': 'mse', 'variance_output': kl_divergence_loss})
    history = model.fit(X, Y, epochs=epochs, batch_size=batch_size, validation_split=0.2, shuffle=True)
    return history, model

def render_plot_loss(history):
  plt.plot(history.history['mean_output_loss'])
  plt.plot(history.history['variance_output_loss'])
  plt.title('model loss')
  plt.ylabel('loss')
  plt.xlabel('epoch')
  plt.legend(['mean', 'variance'], loc='upper left')
  plt.show()

Data preparation:

In [144]:
from sklearn.model_selection import train_test_split

df = pd.read_csv(get_dataframe_path_from_params())

group_on = ['sample_site_lon', 'sample_site_lat', 'month_of_year']
grouped = df.groupby(group_on)

# ASSUMPTION: Taking the mean and variance of a sample site doesn't lower quality of the data.
# We need to do this to use KL-divergence loss.
means = grouped.mean()
O18_var = grouped.var()['cellulose_oxygen_ratio']

# Merging results in some unreadable column names. Rename the oxygen columns.
merged = pd.merge(means, O18_var, on=group_on, how='inner').reset_index()
merged = merged.rename(columns={
    'cellulose_oxygen_ratio_x': 'O18_mean',
    'cellulose_oxygen_ratio_y' : 'O18_var'})

# ...and drop sample_site_lon/sample_site_lat. These were keys used to identify
# sample sites. They are basically duplicates of the 'lat' 'lon' columns.
merged.drop(merged.columns[merged.columns.str.contains('unnamed',case = False)], axis = 1, inplace = True)
merged.drop('sample_site_lon', axis = 1, inplace = True)
merged.drop('sample_site_lat', axis = 1, inplace = True)

train, test = train_test_split(merged, test_size=0.25, random_state=25)

# Target data: Mean and variance
Y_train = train[["O18_mean", "O18_var"]]
Y_test = test[["O18_mean", "O18_var"]]

def format_output(Y):
  y1 = Y.pop("O18_mean")
  y1 = np.array(y1)
  y2 = Y.pop("O18_var")
  y2 = np.array(y2)
  return y1, y2

Y_train = format_output(Y_train)
Y_test = format_output(Y_test)

# Features: Everything besides mean and variance
X_train = train.drop(["O18_mean", "O18_var"], axis=1)
X_test = test.drop(["O18_mean", "O18_var"], axis=1)
X_train.drop(X_train.columns[X_train.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
X_test.drop(X_test.columns[X_test.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

Variance KL-loss: 0.9

Mean MSE loss: 26.37

"General loss": 27.03

In [149]:
# Train
history, model = train_nn(X_train, Y_train, hidden_layers=[12, 20], epochs=5000, batch_size=60)
render_plot_loss(history)

Epoch 1/5000
1/2 [==============>...............] - ETA: 0s - loss: 1468.2520 - mean_output_loss: 1466.0281 - variance_output_loss: 2.2239

2/2 [==============================] - 1s 226ms/step - loss: 1486.2655 - mean_output_loss: 1484.0350 - variance_output_loss: 2.2305 - val_loss: 1420.4620 - val_mean_output_loss: 1417.0760 - val_variance_output_loss: 3.3862
Epoch 2/5000
2/2 [==============================] - 0s 33ms/step - loss: 1483.1398 - mean_output_loss: 1480.3823 - variance_output_loss: 2.7575 - val_loss: 1416.8003 - val_mean_output_loss: 1413.4315 - val_variance_output_loss: 3.3688
Epoch 3/5000
2/2 [==============================] - 0s 55ms/step - loss: 1479.1317 - mean_output_loss: 1476.7300 - variance_output_loss: 2.4017 - val_loss: 1413.1183 - val_mean_output_loss: 1409.7671 - val_variance_output_loss: 3.3513
Epoch 4/5000
2/2 [==============================] - 0s 32ms/step - loss: 1474.8779 - mean_output_loss: 1473.0408 - variance_output_loss: 1.8369 - val_loss: 1409.4141 - val_mean_output_loss: 1406.0804 - val_variance_output_loss: 3.3337
Epoch 5/5000
2/2 [==============================] - 0s 38ms/step - loss:

KeyboardInterrupt: ignored

In [147]:
model.evaluate(x=X_test, y=Y_test)
print(X_test)
print(Y_test)
predictions = model.predict(X_test)
print(predictions)

2/2 [==============================] - 0s 8ms/step - loss: -6.4923 - mean_output_loss: 9.3560 - variance_output_loss: -15.8484
     month_of_year        rh       temp       vpd  atmosphere_oxygen_ratio  \
85               1  0.827577  26.186667  0.602667                -4.758965   
42               6  0.727107  24.515000  0.911667                -2.424168   
43               7  0.698422  25.745000  1.082000                -1.333719   
40               4  0.814435  25.093333  0.617333                -5.834663   
75               3  0.838914  24.296667  0.515333                -7.677537   
93               9  0.759181  27.686667  0.927667                -2.068940   
21               9  0.810539  26.473333  0.681333                -2.441832   
133              1  0.852663  25.540000  0.498333                -4.644259   
103              7  0.752574  26.878333  0.913333                -0.682668   
141              9  0.758108  27.191667  0.916667                -1.753848   
125            

In [117]:
model.save(get_model_save_location(), save_format="h5")

## Generating GeoTIFFs from the DNN

All of the code from the following block is (temporarily) copy and pasted from the library files.

In [118]:
from tensorflow import keras
from dataclasses import dataclass
from osgeo import gdal, gdal_array
from tqdm import tqdm
import math

@dataclass
class AmazonGeoTiff:
  """Represents a geotiff from our dataset."""
  gdal_dataset: gdal.Dataset
  image_value_array: np.ndarray # ndarray of floats
  image_mask_array: np.ndarray # ndarray of uint8
  masked_image: np.ma.masked_array
  yearly_masked_image: np.ma.masked_array


@dataclass
class Bounds:
  """Represents geographic bounds and size information."""
  minx: float
  maxx: float
  miny: float
  maxy: float
  pixel_size_x: float
  pixel_size_y: float
  raster_size_x: float
  raster_size_y: float

def to_matplotlib(self) -> List[float]:
    return [self.minx, self.maxx, self.miny, self.maxy]

def load_raster(path: str, use_only_band_index: int = -1) -> AmazonGeoTiff:
  """
  TODO: Refactor (is_single_band, etc., should be a better design)
  --> Find a way to simplify this logic. Maybe it needs to be more abstract.
  """
  dataset = gdal.Open(path, gdal.GA_ReadOnly)
  image_datatype = dataset.GetRasterBand(1).DataType
  mask_datatype = dataset.GetRasterBand(1).GetMaskBand().DataType
  image = np.zeros((dataset.RasterYSize, dataset.RasterXSize, 12),
                   dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))
  mask = np.zeros((dataset.RasterYSize, dataset.RasterXSize, 12),
                  dtype=gdal_array.GDALTypeCodeToNumericTypeCode(image_datatype))

  if use_only_band_index == -1:
    if dataset.RasterCount != 12 and dataset.RasterCount != 1:
      raise ValueError(f"Expected 12 raster bands (one for each month) or one annual average, but found {dataset.RasterCount}")
    if dataset.RasterCount == 1:
      use_only_band_index = 0

  is_single_band = use_only_band_index != -1

  if is_single_band and use_only_band_index >= dataset.RasterCount:
    raise IndexError(f"Specified raster band index {use_only_band_index}"
                     f" but there are only {dataset.RasterCount} rasters")

  for band_index in range(12):
    band = dataset.GetRasterBand(use_only_band_index+1 if is_single_band else band_index+1)
    image[:, :, band_index] = band.ReadAsArray()
    mask[:, :, band_index] = band.GetMaskBand().ReadAsArray()
  masked_image = np.ma.masked_where(mask == 0, image)
  yearly_masked_image = masked_image.mean(axis=2)

  return AmazonGeoTiff(dataset, image, mask, masked_image, yearly_masked_image)

def get_extent(dataset):
  geoTransform = dataset.GetGeoTransform()
  minx = geoTransform[0]
  maxy = geoTransform[3]
  maxx = minx + geoTransform[1] * dataset.RasterXSize
  miny = maxy + geoTransform[5] * dataset.RasterYSize
  return Bounds(minx, maxx, miny, maxy, geoTransform[1], geoTransform[5], dataset.RasterXSize, dataset.RasterYSize)

def coords_to_indices(bounds: Bounds, x: float, y: float):
  if x < bounds.minx or x > bounds.maxx or y < bounds.miny or y > bounds.maxy:
    raise ValueError("Coordinates out of bounds")

  # X => lat, Y => lon
  x_idx = bounds.raster_size_y - int(math.ceil((y - bounds.miny) / abs(bounds.pixel_size_y)))
  y_idx = int((x - bounds.minx) / abs(bounds.pixel_size_x))

  return x_idx, y_idx

def get_data_at_coords(dataset: AmazonGeoTiff, x: float, y: float, month: int) -> float:
  # x = longitude
  # y = latitude
  bounds = get_extent(dataset.gdal_dataset)
  x_idx, y_idx = coords_to_indices(bounds, x, y)
  if month == -1:
    value = dataset.yearly_masked_image[x_idx, y_idx]
  else:
    value = dataset.masked_image[x_idx, y_idx, month]
  if np.ma.is_masked(value):
    raise ValueError("Coordinates are masked")
  else:
    return value


The following code is new stuff, and used to generate a 12 GeoTIFFs (one for each month) from the model.

In [125]:
def get_predictions_at_each_pixel(
    monthly: bool,
    geotiffs: dict[str, AmazonGeoTiff],
    bounds: Bounds,
    model: keras.Model):
  feature_names = ["lat", "lon", "month_of_year"] + list(geotiffs.keys())
  predicted_means_isoscape = np.ma.array(
      np.zeros([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=float),
      mask=np.ones([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=bool))
  predicted_vars_isoscape = np.ma.array(
      np.zeros([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=float),
      mask=np.ones([bounds.raster_size_x, bounds.raster_size_y, 1], dtype=bool))

  for month in range (0, 12 if monthly else 1):
    for x_idx, x in enumerate(tqdm(np.arange(bounds.minx, bounds.maxx, bounds.pixel_size_x, dtype=float))):
      rows = []
      row_indexes = []
      for y_idx, y in enumerate(np.arange(bounds.miny, bounds.maxy, -bounds.pixel_size_y, dtype=float)):
        row = {}
        try:
          for geotiff_label, geotiff in geotiffs.items():
            row[geotiff_label] = get_data_at_coords(geotiff, x, y, month)
          row["month_of_year"] = month
          row["lon"] = x
          row["lat"] = y
        except ValueError:
          continue # masked and out-of-bounds coordinates
        except IndexError:
          continue
        rows.append(row)
        row_indexes.append((y_idx,month,))
      if (len(rows) > 0):
        X = pd.DataFrame.from_dict(rows)
        predictions = model.predict(X)
        print(predictions)
        means_np = predictions[0]
        for prediction, (y_idx, month_idx) in zip(means_np, row_indexes):
          predicted_means_isoscape.mask[x_idx,y_idx,month_idx] = False # unmask since we have data
          predicted_means_isoscape.data[x_idx,y_idx,month_idx] = prediction
        vars_np = predictions[1]
        for prediction, (y_idx, month_idx) in zip (vars_np, row_indexes):
          predicted_vars_isoscape.mask[x_idx, y_idx, month_idx] = False
          predicted_vars_isoscape.data[x_idx, y_idx, month_idx] = prediction



  return predicted_isoscape

In [126]:
model = keras.models.load_model(get_model_save_location(), custom_objects={"kl_divergence_loss": kl_divergence_loss})

relative_humidity_geotiff = load_raster(get_raster_path_from_params("R.rh_Stack.tif"))
temperature_geotiff = load_raster(get_raster_path_from_params("Temperatura_Stack.tif"))
vapor_pressure_deficit_geotiff = load_raster(get_raster_path_from_params("R.vpd_Stack.tif"))
atmosphere_isoscape_geotiff = load_raster(get_raster_path_from_params("Iso_Oxi_Stack.tif"))

name_to_geotiff = {
    "rh": relative_humidity_geotiff,
    "temp" : temperature_geotiff,
    "vpd" : vapor_pressure_deficit_geotiff,
    "atmosphere_oxygen_ratio" : atmosphere_isoscape_geotiff,
}

# We need the borders of the map. Pick one geotiff at random and use that as the extent.
bounds =  get_extent(atmosphere_isoscape_geotiff.gdal_dataset)

isoscape_np = get_predictions_at_each_pixel(
    monthly=True,
    geotiffs=name_to_geotiff,
    bounds=bounds,
    model=model)





  0%|          | 0/940 [00:00<?, ?it/s]

1/1 [==============================] - 0s 68ms/step


  0%|          | 1/940 [00:00<02:03,  7.59it/s]

[array([[38.533215]], dtype=float32), array([[0.]], dtype=float32)]
1/1 [==============================] - 0s 19ms/step


  0%|          | 2/940 [00:00<01:46,  8.77it/s]

...

  3%|▎         | 24/940 [00:02<01:45,  8.65it/s]

[array([[38.677567],
       [38.66332 ],
       [38.646347],
       [38.6356  ],
       [38.612423],
       [38.504326],
       [38.491432],
       [38.479046],
       [38.459682],
       [38.44137 ],
       [38.425823],
       [38.412487],
       [38.395794],
       [38.38844 ],
       [38.3691  ],
       [38.35431 ],
       [38.342163],
       [38.3326  ],
       [38.304688],
       [38.29232 ],
       [38.29129 ],
       [38.27464 ],
       [38.256035],
       [38.236855],
       [38.222088],
       [38.199364],
       [38.19182 ],
       [38.168835],
       [38.1692  ],
       [38.143097],
       [38.136497],
       [38.132442],
       [38.10302 ],
       [38.095524],
       [38.08636 ],
       [38.073544],
       [38.05473 ],
       [38.036076],
       [38.03139 ],
       [38.01993 ],
       [38.01045 ],
       [37.99655 ],
       [37.98367 ],
       [37.97193 ],
       [37.959583],
       [37.94627 ],
       [37.928493],
       [37.927242],
       [37.918007],
       [37.904057],

  3%|▎         | 25/940 [00:02<01:39,  9.21it/s]

[array([[38.663406],
       [38.65576 ],
       [38.646908],
       [38.62298 ],
       [38.60189 ],
       [38.58276 ],
       [38.500664],
       [38.4855  ],
       [38.466957],
       [38.45594 ],
       [38.436623],
       [38.422382],
       [38.404438],
       [38.400425],
       [38.37347 ],
       [38.365643],
       [38.35435 ],
       [38.342228],
       [38.326164],
       [38.311474],
       [38.28719 ],
       [38.280888],
       [38.259304],
       [38.254257],
       [38.235588],
       [38.219646],
       [38.20394 ],
       [38.185463],
       [38.167267],
       [38.14841 ],
       [38.14953 ],
       [38.121346],
       [38.11383 ],
       [38.107567],
       [38.092052],
       [38.071846],
       [38.06338 ],
       [38.047737],
       [38.02891 ],
       [38.01872 ],
       [38.005898],
       [38.000965],
       [37.98035 ],
       [37.96843 ],
       [37.95562 ],
       [37.943306],
       [37.930893],
       [37.9176  ],
       [37.907196],
       [37.911797],

KeyboardInterrupt: ignored